In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Lambda, Add, LeakyReLU,  \
                                    MaxPooling2D, concatenate, UpSampling2D, Multiply, ZeroPadding2D, Cropping2D
             

In [3]:
%run ./Modules/layer.ipynb
%run ./Modules/activation.ipynb


Exception: File `'./Modules/layer.ipynb'` not found.

In [ ]:

  

def fft_layer(image):
    # get real and imaginary portions
    real = Lambda(lambda image: image[:, :, :, 0])(image)
    imag = Lambda(lambda image: image[:, :, :, 1])(image)

    image_complex = tf.complex(real, imag)  # Make complex-valued tensor
    kspace_complex = tf.signal.fft2d(image_complex)

    # expand channels to tensorflow/keras format
    real = tf.expand_dims(tf.math.real(kspace_complex), -1)
    imag = tf.expand_dims(tf.math.imag(kspace_complex), -1)
    kspace = tf.concat([real, imag], -1)
    return kspace



def ifft_layer(kspace_2channel):
    # Get real and imaginary parts
    real = Lambda(lambda kspace_2channel: kspace_2channel[:, :, :, 0])(kspace_2channel)
    imag = Lambda(lambda kspace_2channel: kspace_2channel[:, :, :, 1])(kspace_2channel)

    kspace_complex = tf.complex(real, imag)  # Convert to complex tensor
    image_complex = tf.signal.ifft2d(kspace_complex)

    # Expand channels for Keras format
    real = tf.expand_dims(tf.math.real(image_complex), -1)
    imag = tf.expand_dims(tf.math.imag(image_complex), -1)

    # Generate 2-channel complex image
    image_complex_2channel = tf.concat([real, imag], -1)

    # Print shape
    #print("Shape of image_complex_2channel:", image_complex_2channel.shape)

    # Compute magnitude for each sample
    magnitude = tf.sqrt(real ** 2 + imag ** 2)

    # Compute min and max per image
    min_per_sample = tf.reduce_min(magnitude, axis=[1, 2], keepdims=True)
    max_per_sample = tf.reduce_max(magnitude, axis=[1, 2], keepdims=True)

    # Print min and max values before normalization
    #tf.print("Min values before normalization (first 5 images):", min_per_sample[:5, 0, 0, 0])
    #tf.print("Max values before normalization (first 5 images):", max_per_sample[:5, 0, 0, 0])

    # Normalize: Ensure max magnitude per image is 1
    max_per_sample = tf.where(max_per_sample == 0, tf.ones_like(max_per_sample), max_per_sample)  # Avoid division by zero
    real /= max_per_sample
    imag /= max_per_sample

    # Recompute magnitude after normalization
    magnitude_after = tf.sqrt(real ** 2 + imag ** 2)
    min_after = tf.reduce_min(magnitude_after, axis=[1, 2], keepdims=True)
    max_after = tf.reduce_max(magnitude_after, axis=[1, 2], keepdims=True)

    # Print min and max values after normalization
    #tf.print("Min values after normalization (first 5 images):", min_after[:5, 0, 0, 0])
    #tf.print("Max values after normalization (first 5 images):", max_after[:5, 0, 0, 0])

    # Reconstruct normalized 2-channel complex image
    image_complex_2channel = tf.concat([real, imag], -1)

    return image_complex_2channel

In [ ]:
def DC_block(rec,mask,sampled_kspace,channels,kspace = False):
    """
    :param rec: Reconstructed data, can be k-space or image domain
    :param mask: undersampling mask
    :param sampled_kspace:
    :param kspace: Boolean, if true, the input is k-space, if false it is image domain
    :return: k-space after data consistency
    """

    if kspace:
        rec_kspace = rec
    else:
        rec_kspace = Lambda(fft_layer)(rec)
    mask = 1 - mask
    rec_kspace_dc =  Multiply()([rec_kspace,mask])
    rec_kspace_dc = Add()([rec_kspace_dc,sampled_kspace])
    return rec_kspace_dc


In [4]:
from keras.layers import Conv2D, LeakyReLU, Add
from keras import backend as K
def cnn_block(cnn_input, nf, kshape, channels):
    """
    CNN block with fixed depth of 5.
    Applies 5 convolutional layers followed by a final 1x1 convolution.
    
    :param cnn_input: Input layer to CNN block
    :param nf: Number of filters in convolutional layers
    :param kshape: Shape of the convolutional kernel
    :param channels: Number of output channels (2 for real and imaginary)
    :return: 2-channel complex reconstruction with residual connection
    """
    
    # Separate real and imaginary parts
    real = cnn_input[..., 0]  # First channel (real part)
    imag = cnn_input[..., 1]  # Second channel (imaginary part)
    #print("real",real.shape)
    #print("imag",imag.shape)
    real = K.expand_dims(real, axis=-1)  # Shape becomes (None, 256, 256, 1)
    imag = K.expand_dims(imag, axis=-1)  # Shape becomes (None, 256, 256, 1)
    # Print the new shapes
    #print("real after", real.shape)  # Should print (None, 256, 256, 1)
    #print("imag after", imag.shape)


    # First convolution and activation
    real_conv1, imag_conv1 = complex_Conv2D(nf, kshape,  padding="same")(real,imag)
    real_conv1, imag_conv1 =CLeaky_ReLU(real_conv1, imag_conv1)
    
    real_conv1, imag_conv1  = complex_Conv2D(nf, kshape, padding='same')(real_conv1, imag_conv1 )
    real_conv1, imag_conv1 =CLeaky_ReLU(real_conv1, imag_conv1)
    
    real_conv1, imag_conv1 = complex_Conv2D(nf, kshape,  padding="same")(real_conv1,imag_conv1)
    real_conv1, imag_conv1 =CLeaky_ReLU(real_conv1, imag_conv1)
    
    real_conv1,imag_conv1=add_with( real_conv1, imag_conv1,real,imag)
    
    real_conv1, imag_conv1 = complex_Conv2D(nf, kshape,  padding="same")(real_conv1,imag_conv1)
    real_conv1, imag_conv1 =CLeaky_ReLU(real_conv1, imag_conv1)
    
    
    real_conv1, imag_conv1 = complex_Conv2D(nf, kshape,  padding="same")(real_conv1,imag_conv1)
    real_conv1, imag_conv1 =CLeaky_ReLU(real_conv1, imag_conv1)
    
    #real_conv1, imag_conv1 = complex_Conv2D(nf, kshape,  padding="same")(real_conv1,imag_conv1)
    #real_conv1, imag_conv1 =CLeaky_ReLU(real_conv1, imag_conv1)
    real_conv1,imag_conv1=add_with( real_conv1, imag_conv1,real,imag)


    # Final 1x1 convolution to return to 2 channels
    real_fianl, imag_final = complex_Conv2D(2, (1, 1))(real_conv1, imag_conv1 )

    # Add residual connection (input + CNN output)
    #real_res1,imag_res1=add_with(real_fianl, imag_final,real,imag)
    res1 =concatenate([real_fianl,imag_final],axis=-1)
    
    return res1


SyntaxError: invalid syntax (925539221.py, line 23)

In [ ]:
from keras.layers import Input, Lambda
from keras.models import Model

def deep_cascade_flat_unrolled(depth_str='ikikii', H=256, W=256, kshape=(3, 3), nf=48, channels=2):
    """
    Simplified Deep Cascade Flat Unrolled model with fixed CNN depth of 5.
    
    :param depth_str: String defining the sequence of IFFT, CNN, and DC operations
    :param H: Image height
    :param W: Image width
    :param kshape: Kernel size for CNN
    :param nf: Number of filters in CNN
    :param channels: Number of input/output channels (real + imaginary = 2)
    :return: Deep Cascade Flat Unrolled Keras model
    """

    # Define inputs
    inputs = Input(shape=(H, W, channels))
   # Input k-space data
    mask = Input(shape=(H, W, channels))   # Sampling mask
    
    # Start with input layer
    x = inputs
    kspace_flag = True

    # Iterate through depth_str to build the cascade
    for ii in depth_str:
        
        if ii == 'i':  # Apply IFFT if 'i'
            x = Lambda(ifft_layer)(x)
            kspace_flag = False
        #print("x",x.shape)
        # Apply CNN block (fixed depth of 5 in simplified version)
        x = cnn_block(x, nf, kshape, channels)

        # Apply DC block for data consistency
        x = DC_block(x, mask, inputs, channels, kspace=kspace_flag)
        kspace_flag = True  # Return to k-space after DC block

    # Final IFFT to return to spatial domain
    out = Lambda(ifft_layer)(x)

    # Define and return the model
    model = Model(inputs=[inputs, mask], outputs=out)
    return model
